# Dawid-Skene Algorithm using the diagonal of the confucion matrix

if we use the diagonal of the confusion matrix $\pi$ we assume:
- annotators only have a probabilty of labeling the correct class correctly


In [210]:
import warnings
from os import PathLike
from sys import getsizeof
from typing import Annotated, Generator

import numpy as np
import sparse as sp
from annotated_types import Ge
from loguru import logger
from memory_profiler import profile
from pydantic import validate_call
from tqdm.auto import tqdm

from peerannot.models.template import CrowdModel,AnswersDict

# load data
from toy_data import votes, N_CLASSES, N_WORKERS



In [252]:

FilePathInput = PathLike | str | list[str] | Generator[str, None, None] | None


class DawidSkeneDiagonal(CrowdModel):
    """
    =============================
    Dawid and Skene model (1979)
    =============================

    Assumptions:
    - independent workers

    Using:
    - EM algorithm

    Estimating:
    - One confusion matrix for each workers
    """

    @validate_call
    def __init__(
        self,
        answers: AnswersDict,
        # TODO@jzftran probably annotation with 0 or 1 worker doesn't make sense: for 0 it should be an error
        n_workers: Annotated[int, Ge(1)],
        n_classes: Annotated[int, Ge(1)],
        *,
        sparse: bool = False,
        path_remove: FilePathInput = None,
    ) -> None:
        r"""Dawid and Skene strategy: estimate confusion matrix for each worker.

        Assuming that workers are independent, the model assumes that

        .. math::

            (y_i^{(j)}\ | y_i^\\star = k) \\sim \\mathcal{M}\\left(\\pi^{(j)}_{k,\\cdot}\\right)

        and maximizes the log likelihood of the model using an EM algorithm.

        .. math::

            \\underset{\\rho,\\\pi,T}{\mathrm{argmax}}\\prod_{i\\in [n_{\\texttt{task}}]}\prod_{k \\in [K]}\\bigg[\\rho_k\prod_{j\\in [n_{\\texttt{worker}}]}\prod_{\\ell\in [K]}\\big(\\pi^{(j)}_{k, \\ell}\\big)^{\mathbf{1}_{\\{y_i^{(j)}=\\ell\\}}}\\bigg]^{T_{i,k}},

        where :math:`\\rho` is the class marginals, :math:`\\pi` is the confusion matrix and :math:`T` is the indicator variables of belonging to each class.

        :param answers: Dictionary of workers answers with format

         .. code-block:: javascript

            {
                task0: {worker0: label, worker1: label},
                task1: {worker1: label}
            }

        :type answers: dict
        :param sparse: If the number of workers/tasks/label is large (:math:`>10^{6}` for at least one), use sparse=True to run per task
        :type sparse: bool, optional
        :param n_classes: Number of possible classes, defaults to 2
        :type n_classes: int, optional"""

        super().__init__(answers)
        self.n_workers: int = n_workers
        self.n_classes: int = n_classes
        self.sparse: bool = sparse
        self.path_remove: FilePathInput = path_remove
        self.n_task: int = len(self.answers)

        self.exclude_answers()
        if self.sparse:
            raise NotImplementedError('The sparse algorithm for DawidSkeneDiagonal in the current context is not implemented.')
        else:
            self.init_crowd_matrix()

    def exclude_answers(self) -> None:
        answers_modif = {}
        if self.path_remove is not None:
            to_remove = np.loadtxt(self.path_remove, dtype=int)
            i = 0
            for key, val in self.answers.items():
                if int(key) not in to_remove[:, 1]:
                    answers_modif[i] = val
                    i += 1
            self.answers = answers_modif


    def init_crowd_matrix(self)->None:
        """Transform dictionnary of labels to a tensor of size (n_task, n_workers, n_classes)."""

        matrix = np.zeros((self.n_task, self.n_workers, self.n_classes))
        for task, ans in self.answers.items():
            for worker, label in ans.items():
                matrix[task, worker, label] += 1

        logger.debug(f"Dense crowd matrix  {getsizeof(matrix)}")
        # shape
        self.crowd_matrix = matrix

    def init_T(self):
        """NS initialization"""
        # shape (n_task, n_workers)
        T = self.crowd_matrix.sum(axis=1)
        logger.debug(f"Size of T before calc: {getsizeof(T)}")

        tdim = T.sum(1, keepdims=True)
        self.T = np.where(tdim > 0, T / tdim, 0)
        logger.debug(f"Size of T: {getsizeof(self.T)}")



    def _m_step(self) -> None:
        """Maximizing log likelihood with only diagonal elements of pi."""
        rho = self.T.sum(0) / self.n_task  

        pi = np.zeros((self.n_workers, self.n_classes))  
        for j in range(self.n_classes):
            # TODO:@jzftran change this to calculate diagonal faster
            pij = self.T[:, j] @ self.crowd_matrix.transpose((1, 0, 2))
            denom = pij.sum(1)

            
            diag_values = pij[:, j] / np.where(denom > 0, denom, 1e-9)
            pi[:, j] = diag_values

        # pi shape (n_workers, n_class), reresents how sure worker is sure that the label j is true

        self.rho, self.pi = rho, pi 


    def _e_step(self) -> None:
        """Estimate indicator variables (see eq. 2.5 Dawid and Skene 1979)

        Returns:
            T: New estimate for indicator variables (n_task, n_worker)
            denom: value used to compute likelihood easily
        """
        T = np.zeros((self.n_task, self.n_classes))
        for i in range(self.n_task):
            for j in range(self.n_classes):
                num = (
                    np.prod(
                        # pi[:,j] should represent how sure are workers that the j-th tasks belond got
                        np.power(self.pi[:, j], self.crowd_matrix[i, :,j])
                    )
                    * self.rho[j]
                )
                T[i, j] = num
        self.denom_e_step = T.sum(1, keepdims=True)
        T = np.where(self.denom_e_step > 0, T / self.denom_e_step, T)
        self.T = T
        
    def log_likelihood(self):
        """Compute log likelihood of the model"""
        return np.log(np.sum(self.denom_e_step))

    def run_dense(
        self,
        epsilon: Annotated[float, Ge(0)] = 1e-6,
        maxiter: Annotated[int, Ge(0)] = 50,
        *,
        verbose: bool = False,
    ) -> tuple[list[np.float64], int]:
        i = 0
        eps = np.inf

        self.init_T()
        ll = []
        pbar = tqdm(total=maxiter, desc="Dawid and Skene")
        while i < maxiter and eps > epsilon:
            self._m_step()
            self._e_step()
            likeli = self.log_likelihood()
            ll.append(likeli)
            if len(ll) >= 2:
                eps = np.abs(ll[-1] - ll[-2])
            i += 1
            pbar.update(1)

        pbar.set_description("Finished")
        pbar.close()
        self.c = i
        if eps > epsilon and verbose:
            print(f"DS did not converge: err={eps}")
        return ll, i


    @validate_call
    def run(
        self,
        epsilon: Annotated[float, Ge(0)] = 1e-6,
        maxiter: Annotated[int, Ge(0)] = 50,
        *,
        verbose: bool = False,
    ) -> tuple[list[np.float64], int]:
        """Run the EM optimization

        :param epsilon: stopping criterion (:math:`\\ell_1` norm between two iterates of log likelihood), defaults to 1e-6
        :type epsilon: float, optional
        :param maxiter: Maximum number of steps, defaults to 50
        :type maxiter: int, optional
        :param verbose: Verbosity level, defaults to False
        :type verbose: bool, optional
        :return: Log likelihood values and number of steps taken
        :rtype: (list,int)
        """

        if self.sparse:
            raise NotImplementedError('The sparse algorithm for DawidSkeneShared in the current context is not implemented.')
        return self.run_dense(
            epsilon=epsilon,
            maxiter=maxiter,
            verbose=verbose,
        )

    def get_answers(self):
        """Get most probable labels"""
        if self.sparse:
            return np.vectorize(self.inv_labels.get)(
                sp.argmax(self.sparse_T, axis=1).todense()
            )
        return np.vectorize(self.inv_labels.get)(
            np.argmax(self.get_probas(), axis=1)
        )

    def get_probas(self):
        """Get soft labels distribution for each task"""
        if self.sparse:
            warnings.warn("Sparse implementation only returns hard labels")
            return self.get_answers()
        return self.T


In [253]:
dsd = DawidSkeneDiagonal(answers=svotes, n_workers=sN_WORKERS, n_classes=sN_CLASSES, sparse=False)
dsd.run(maxiter=500)
dsd.get_answers()

2025-03-20 16:45:14.992 | INFO     | __main__:init_crowd_matrix:95 - Dense crowd matrix  336
2025-03-20 16:45:14.993 | INFO     | __main__:init_T:103 - Size of T before calc: 192
2025-03-20 16:45:14.994 | INFO     | __main__:init_T:107 - Size of T: 192
















Finished:   2%|▏         | 10/500 [00:00<00:00, 3927.25it/s]


array([1, 0, 0, 0])

In [254]:
from peerannot.models.aggregation.DS import DawidSkene
from simple_toy_data import votes as svotes, N_WORKERS as sN_WORKERS, N_CLASSES as sN_CLASSES
ds = DawidSkene(answers=svotes, n_workers=sN_WORKERS, n_classes=sN_CLASSES, sparse=False)
ds.run(maxiter=500)
ds.get_answers()

2025-03-20 16:45:16.901 | INFO     | peerannot.models.aggregation.DS:init_crowd_matrix:127 - Dense crowd matrix  336
2025-03-20 16:45:16.902 | INFO     | peerannot.models.aggregation.DS:__init__:92 - Dense Crowd matrix336
2025-03-20 16:45:16.903 | INFO     | peerannot.models.aggregation.DS:init_T:134 - Size of T before calc: 192
2025-03-20 16:45:16.904 | INFO     | peerannot.models.aggregation.DS:init_T:138 - Size of T: 192
















Finished:   2%|▏         | 8/500 [00:00<00:00, 3141.80it/s]


array([0, 1, 1, 1])